In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

In [ ]:

!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)



cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from selenium.webdriver.common.keys import Keys
import os
os.chdir("/content/drive/MyDrive")

In [ ]:
search_URL = ["surprised face human images","angry face human images","disguist face human images","fear face human images","happy face human images","neutral face human images","sad face human images"]

In [ ]:
# search_URL = "https://www.google.com/search?q=surprised+human+images&tbm=isch&ved=2ahUKEwijhoucy5j4AhVLdJQKHbUzBeMQ2-cCegQIABAA&oq=surprised+human+images&gs_lcp=CgNpbWcQAzIGCAAQHhAHOgQIABATOggIABAeEAcQEzoICAAQHhAIEBNQrgRY4Bxg6B9oA3AAeACAAYoBiAHWBpIBAzQuNJgBAKABAaoBC2d3cy13aXotaW1nwAEB&sclient=img&ei=6dKdYuPtG8vo0QS155SYDg&rlz=1C1CHBF_enVN822VN822"
# driver.get(search_URL)

In [ ]:
import bs4
import os
import time
import requests


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
def download_image(url, folder_name, num):
    # write image to file
    reponse = requests.get(url)
    if reponse.status_code==200:
        with open(os.path.join(folder_name, str(num)+".jpg"), 'wb') as file:
            file.write(reponse.content)

def scroll_to_bottom():
 
    last_height = driver.execute_script('\
    return document.body.scrollHeight')
 
    while True:
        driver.execute_script('\
        window.scrollTo(0,document.body.scrollHeight)')
 
        # waiting for the results to load
        # Increase the sleep time if your internet is slow
        time.sleep(3)
 
        new_height = driver.execute_script('\
        return document.body.scrollHeight')
 
        # click on "Show more results" (if exists)
        try:
            driver.find_element_by_css_selector(".YstHxe input").click()
 
            # waiting for the results to load
            # Increase the sleep time if your internet is slow
            time.sleep(3)
 
        except:
            pass
 
        # checking if we have reached the bottom of the page
        if new_height == last_height:
            break
 
        last_height = new_height
 
 
# Calling the function
 
# NOTE: If you only want to capture a few images,
# there is no need to use the scroll_to_bottom() function.


In [ ]:
# # driver.execute_script("window.scrollTo(0, 0);")
# folder_name = './data/'+query.split(" ",0)
# if not os.path.isdir(folder_name):
#     os.makedirs(folder_name)
# # driver.get(search_URL)
# driver.maximize_window()
 
# # Open Google Images in the browser
# driver.get('https://images.google.com/')
 
# # Finding the search box
# box = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
 
# # Type the search query in the search box
# box.send_keys(query)
 
# # Pressing enter
# box.send_keys(Keys.ENTER)
# scroll_to_bottom()
# page_html = driver.page_source
# pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
# containers = pageSoup.findAll('div', {'class':"bRMDJf islir"} )

# print(len(containers))

In [ ]:
for query in search_URL:
  folder_name = './data/'+query.split(" ")[0]
  if not os.path.isdir(folder_name):
      os.makedirs(folder_name)
  # driver.get(search_URL)
  driver.maximize_window()
  
  # Open Google Images in the browser
  driver.get('https://images.google.com/')
  
  # Finding the search box
  box = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
  
  # Type the search query in the search box
  box.send_keys(query)
  
  # Pressing enter
  box.send_keys(Keys.ENTER)
  scroll_to_bottom()
  page_html = driver.page_source
  pageSoup = bs4.BeautifulSoup(page_html, 'html.parser')
  containers = pageSoup.findAll('div', {'class':"bRMDJf islir"} )

  print(len(containers))
  len_containers = len(containers)

  for i in range(1, len_containers+1):
      if i % 25 == 0:
          continue

      xPath = """//*[@id="islrg"]/div[1]/div[%s]"""%(i)

      previewImageXPath = """//*[@id="islrg"]/div[1]/div[%s]/a[1]/div[1]/img"""%(i)
      previewImageElement = driver.find_element_by_xpath(previewImageXPath)
      previewImageURL = previewImageElement.get_attribute("src")





      driver.find_element_by_xpath(xPath).click()
  

      timeStarted = time.time()
      while True:
          try:
            imageElement = driver.find_element_by_xpath("""//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img""")
          except:
            break
          imageURL= imageElement.get_attribute('src')

          if imageURL != previewImageURL:
              #print("actual URL", imageURL)
              break

          else:
              #making a timeout if the full res image can't be loaded
              currentTime = time.time()

              if currentTime - timeStarted > 10:
                  print("Timeout! Will download a lower resolution image and move onto the next one")
                  break


      #Downloading image
      try:
          download_image(imageURL, folder_name, i)
          print("Downloaded element %s out of %s total. URL: %s" % (i, len_containers + 1, imageURL))
      except:
          print("Couldn't download an image %s, continuing downloading the next one"%(i))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


803


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead


Downloaded element 1 out of 804 total. URL: https://thumbs.dreamstime.com/b/happy-young-attractive-woman-shocked-surprised-funny-face-human-expressions-emotions-beautiful-blond-pointing-making-154069272.jpg
Downloaded element 2 out of 804 total. URL: https://thumbs.dreamstime.com/b/happy-young-attractive-teenager-girl-shocked-surprised-funny-face-human-expressions-beautiful-blonde-woman-making-gestures-146997332.jpg
Downloaded element 3 out of 804 total. URL: https://image.shutterstock.com/z/stock-photo-human-face-expressions-and-emotions-african-male-screaming-in-surprise-mouth-wide-open-holding-492174064.jpg
Downloaded element 4 out of 804 total. URL: https://thumbs.dreamstime.com/z/happy-young-attractive-woman-shocked-surprised-funny-face-human-expressions-emotions-beautiful-blond-pointing-making-154069182.jpg
Downloaded element 5 out of 804 total. URL: https://i.pinimg.com/originals/6d/fb/56/6dfb5660c821203ebfc479951993a0a3.jpg
Downloaded element 6 out of 804 total. URL: https://th

NoSuchElementException: ignored